# Perth Housing Analysis

In [1]:
# Import dependencies 

import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [2]:
# Import CSV 
perth_housing = pd.read_csv('Perth_housing_data.csv')
perth_housing.head()

,ADDRESS,SUBURB,PRICE,BEDROOMS,BATHROOMS,GARAGE,LAND_AREA,FLOOR_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST,NEAREST_SCH_RANK
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,-32.193470,115.859554,ATWELL COLLEGE,5.524324,129.0
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,-31.885790,115.947780,KIARA COLLEGE,1.514922,NaN


## Clean the data ## 


In [3]:
# Delete irrelevant columsn = garage, address, floor area.
new_perth_housing = perth_housing.drop(['ADDRESS', 'GARAGE', 'FLOOR_AREA', 'NEAREST_SCH_RANK'] , axis =1)

# Remove all null values including
new_perth_housing.dropna(how ='any', inplace = True)

# Convert School distance to meters and convert to integer
new_perth_housing['NEAREST_SCH_DIST'] = (new_perth_housing[['NEAREST_SCH_DIST']] * 1000).astype(int)

#Convert Build Year to Integer 
new_perth_housing['BUILD_YEAR'] = new_perth_housing['BUILD_YEAR'].astype(int)

# Remove post codes which are out of Perth
new_perth_housing = new_perth_housing[new_perth_housing['POSTCODE'] < 6200]

In [4]:
#Check the properties of the dataframe. 
new_perth_housing.shape

(30378, 15)

In [5]:
# Change datatype of DATE SOLD column to string to remove '/r' 
new_perth_housing[['DATE_SOLD']] = new_perth_housing[['DATE_SOLD']].astype('string')
new_perth_housing['DATE_SOLD'] = new_perth_housing['DATE_SOLD'].str[:7]

#Split Month and Year 
new_perth_housing['DATE_SOLD'] = pd.to_datetime(new_perth_housing['DATE_SOLD'])

# Add clean data to new frame
clean_perth_housing = new_perth_housing

C:\Users\jesss\AppData\Local\Temp\ipykernel_18324\868040009.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  new_perth_housing['DATE_SOLD'] = pd.to_datetime(new_perth_housing['DATE_SOLD'])


In [6]:
clean_perth_housing.head(5)

,SUBURB,PRICE,BEDROOMS,BATHROOMS,LAND_AREA,BUILD_YEAR,CBD_DIST,NEAREST_STN,NEAREST_STN_DIST,DATE_SOLD,POSTCODE,LATITUDE,LONGITUDE,NEAREST_SCH,NEAREST_SCH_DIST
0,South Lake,565000,4,2,600,2003,18300,Cockburn Central Station,1800,2018-09-01,6164,-32.115900,115.842450,LAKELAND SENIOR HIGH SCHOOL,828
1,Wandi,365000,3,2,351,2013,26900,Kwinana Station,4900,2019-02-01,6167,-32.193470,115.859554,ATWELL COLLEGE,5524
2,Camillo,287000,3,1,719,1979,22600,Challis Station,1900,2015-06-01,6111,-32.120578,115.993579,KELMSCOTT SENIOR HIGH SCHOOL,1649
3,Bellevue,255000,2,1,651,1953,17900,Midland Station,3600,2018-07-01,6056,-31.900547,116.038009,SWAN VIEW SENIOR HIGH SCHOOL,1571
4,Lockridge,325000,4,1,466,1998,11200,Bassendean Station,2000,2016-11-01,6054,-31.885790,115.947780,KIARA COLLEGE,1514


In [8]:
clean_perth_housing.to_csv("clean_perth_data.csv", header=True, index =False)